In [54]:
import requests
import pandas as pd
import numpy as np
from coinbase.rest import RESTClient
import datetime
import json
import time
import os
import plotly.express as px



In [55]:

count = 1000

tstart = int(time.time()) - 60*60 # a minute ago

api_key = os.environ.get("COINBASE_API_TOKEN")
api_secret = os.environ.get("COINBASE_PRIVATE_TOKEN")

client = RESTClient(api_key=api_key, api_secret=api_secret)
prod = "MATIC-USD"

coinbase = client.get_public_market_trades(prod,1000,start=tstart)

for ii in range(len(coinbase['trades'])):
    coinbase['trades'][ii] = coinbase['trades'][ii].__dict__


In [56]:

cb = pd.DataFrame.from_dict(coinbase['trades'])
cb
# cb['time'] = pd.to_datetime(cb['time'])
cb['time'] = cb['time'].apply(lambda x: float(datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()))
cb['time'] = cb['time'] - 5*60*60 # EST TO UTC
cb['price'] = cb['price'].astype(float)
cb['size'] = cb['size'].astype(float)
cb = cb.set_index('time')
cb = cb.drop(['product_id','trade_id', 'exchange', 'bid', 'ask'], axis=1)
cb

,price,size,side
time,,,
1.734634e+09,0.4779,197.4,BUY
1.734634e+09,0.4779,26.3,BUY
1.734634e+09,0.4785,21.9,SELL
1.734634e+09,0.4787,40.6,SELL
1.734634e+09,0.4782,150.9,BUY
1.734634e+09,0.4782,1376.9,SELL
1.734634e+09,0.4783,153.0,SELL
1.734634e+09,0.4761,22.0,BUY
1.734634e+09,0.4758,210.0,BUY


In [57]:

url = "https://api.kraken.com/0/public/Trades?pair={}&since={}".format("MATICUSD",tstart)
print(url)

payload = {}
headers = {
  'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

m = json.loads(response.text)
kk = pd.DataFrame(m['result']['MATICUSD'])
kk
kk.columns=['price','size', 'time', 'side', 'idk', 'type', 'ind']
kk
kk = kk.drop(['ind', 'idk', 'type'], axis=1)

kk['time'] = kk['time'].astype(float)
kk['price'] = kk['price'].astype(float)
kk['size'] = kk['size'].astype(float)
kk = kk.set_index('time')
kk['side'] = kk['side'].apply(lambda s: 'SELL' if s == 's' else 'BUY')
kk

https://api.kraken.com/0/public/Trades?pair=MATICUSD&since=1734633962


,price,size,side
time,,,
1.734634e+09,0.4764,33.444070,SELL
1.734634e+09,0.4762,231.577319,BUY
1.734634e+09,0.4763,768.471302,BUY
1.734634e+09,0.4784,38.015888,SELL
1.734634e+09,0.4783,73.561292,SELL


In [58]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=cb.index, y=cb['price'], mode='lines', name='Coinbase'))
fig.add_trace(go.Scatter(x=kk.index, y=kk['price'], mode='lines', name='Kraken'))
fig.show()


56.84140110015869